In [ ]:
#######匯入
import requests
import urlparse
from bs4 import BeautifulSoup
from math import ceil

In [ ]:
#######建立session
payload = {
'method':'search',
'searchMethod':'true',
'tenderUpdate':'',
'searchTarget':'',
'orgName':'',
'orgId':'',
'hid_1':'1',
'tenderName':'',
'tenderId':'',
'tenderType':'tenderDeclaration',
'tenderWay':'1,2,3,4,5,6,7,10,12',
'tenderDateRadio':'on',
'tenderStartDateStr':'105/02/13',
'tenderEndDateStr':'105/02/19',
'tenderStartDate':'105/02/19',
'tenderEndDate':'105/02/19',
'isSpdt':'N',
'proctrgCate':'3',
'radProctrgCate':'3',
'btnQuery':'查詢',
'hadUpdated':'',
}
rs = requests.session()
user_post = rs.post('http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=1',data=payload)
response_text = user_post.text.encode('utf8')

In [ ]:
#######計算抓取頁數
soup = BeautifulSoup(response_text)
rec_number_element = soup.select('.T11b')[0].text
page_number = int(ceil(float(rec_number_element)/100))
print page_number

In [ ]:
#######存入每頁標案連結
bid_file = open('bid_list.txt','w')
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=%d"
###建立第一層迴圈用來翻頁P1~P6
for page in range(1, page_number + 1):
###進入頁面
    bid_list = rs.get(page_format%(page))
    bid_response = bid_list.text.encode('utf8')
    bid_soup = BeautifulSoup(bid_response)
###建立第二層迴圈存入各行連結,第三層迴圈將地址存到link裡面
    bid_div = bid_soup.select('#print_area')[0]
    bid_rows =  bid_div.select('tr')[1:-1]
    for bid_row in bid_rows:
        link = [tag['href'] for tag in bid_row.select('a')][0]
        link_href = urlparse.urljoin('http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex=',link)
        bid_file.write(link_href + '\n')
bid_file.close()

In [ ]:
#######抓取標案細節完整版
from bs4 import BeautifulSoup
import re, requests, time
bid_list = open('bid_list.txt', 'r')
###迴圈進入bid_list進行讀取
for line in bid_list.readlines():
###pagelink是每條網址,用strip()切斷
    pagelink = line.strip()
###m的功能用在group兩個PK----本ETL未使用可整行註解
    m = re.match(r"([^ ]+)&primaryKey=(?P<primaryKey>.+)", pagelink)
    filename = pagelink.split('primaryKey=')[1]
    request_get = requests.get(pagelink)
    response = request_get.text.encode('utf8')
    soup = BeautifulSoup(response)
    printarea = soup.select('#print_area')[0]
    bid_detail = open('/Users/Gsr00001/Desktop/ETLhomework/{}.text'.format(filename),'w')
    bid_detail.write(printarea.prettify().encode('utf-8'))
###睡眠時間3用來閃避網頁判讀
    time.sleep(3)
    bid_detail.close()
bid_list.close()